# Exprole multi backend Keras - Keras 3.0

## Install dependencies

In [1]:
! pip install -q keras==3.0

## Set the backend
In this tutorial we will use Tensorflow as our backend, you can change it to "jax" or "torch" for JAX and PyTorch separately.

In [2]:
import os

# Try picking a different value from "tensorflow", "jax" and "torch".
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import tensorflow as tf
import keras

import numpy as np

## Use a Keras layer

Layers are callable

In [4]:
layer = keras.layers.Dense(
    2,
    activation="relu",
    bias_initializer="zeros",
)
data = np.random.uniform(size=[3, 3])
layer(data)

tensor([[0.0000, 0.0000],
        [0.2381, 0.4862],
        [0.1233, 0.3835]], grad_fn=<ReluBackward0>)

## Write your custom layer

In [5]:
class Linear(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=[input_dim, self.units], initializer="glorot_uniform",
        )
        self.b = self.add_weight(
            shape=[self.units,], initializer="zeros"
        )

    def call(self, inputs):
        return keras.ops.matmul(inputs, self.w) + self.b

layer = Linear(2)
layer(data)

tensor([[-0.2625,  0.5636],
        [-0.9150,  1.2975],
        [-0.7664,  1.1620]], grad_fn=<AddBackward0>)

## Different ways of building the model

*  Sequential API
*  [Preferred] Functional API
*   Subclassing model

In this demo, we are building a 2-class CNN image classifier.

In [6]:
# Sequential model
sequential_model = keras.Sequential([
    keras.Input([28, 28, 3]),
    keras.layers.Conv2D(8, 2),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(2),
    keras.layers.Softmax(),
])

print(sequential_model.summary())
print(sequential_model(np.random.uniform(size=[1, 28, 28, 3])))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                    │ (None, 27, 27, 8)             │         104 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ max_pooling2d (MaxPooling2D)       │ (None, 13, 13, 8)             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ flatten (Flatten)                  │ (None, 1352)                  │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_1 (Dense)                    │ (None, 2)                     │       2,706 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ softmax (Softmax)                  │ (None, 2)                     │           0 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None
tensor([[0.3802, 0.6198]], grad_fn=<SoftmaxBackward0>)


In [7]:

# Functional model
inputs = keras.Input([28, 28, 3])
x = keras.layers.Conv2D(8, 2)(inputs)
x = keras.layers.MaxPool2D(2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(functional_model.summary())
print(functional_model(np.random.uniform(size=[1, 28, 28, 3])))

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)         │ (None, 28, 28, 3)             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ conv2d_1 (Conv2D)                  │ (None, 27, 27, 8)             │         104 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ max_pooling2d_1 (MaxPooling2D)     │ (None, 13, 13, 8)             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ flatten_1 (Flatten)                │ (None, 1352)                  │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_2 (Dense)                    │ (None, 2)                     │       2,706 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ softmax_1 (Softmax)                │ (None, 2)                     │           0 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None
tensor([[0.5268, 0.4732]], grad_fn=<SoftmaxBackward0>)


In [8]:
# Multi-input model.
input_1 = keras.Input([28, 28, 3])
input_2 = keras.Input([28, 28, 3])
x_1 = keras.layers.Conv2D(8, 2)(input_1)
x_2 = keras.layers.Conv2D(8, 2)(input_2)
x = keras.layers.MaxPool2D(2)(x_1 + x_2)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

multi_input_functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(multi_input_functional_model.summary())

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃    Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 28, 28, 3)      │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ input_layer_3             │ (None, 28, 28, 3)      │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 27, 27, 8)      │        104 │ input_layer_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 27, 27, 8)      │        104 │ input_layer_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ add (Add)                 │ (None, 27, 27, 8)      │          0 │ conv2d_2[0][0],            │
│                           │                        │            │ conv2d_3[0][0]             │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ max_pooling2d_2           │ (None, 13, 13, 8)      │          0 │ add[0][0]                  │
│ (MaxPooling2D)            │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 1352)           │          0 │ max_pooling2d_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ dense_3 (Dense)           │ (None, 2)              │      2,706 │ flatten_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ input_layer_1             │ (None, 28, 28, 3)      │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ softmax_2 (Softmax)       │ (None, 2)              │          0 │ dense_3[0][0]              │
└───────────────────────────┴────────────────────────┴────────────┴────────────────────────────┘

 Total params: 2,914 (11.38 KB)

 Trainable params: 2,914 (11.38 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
# Subclassing `keras.Model`

class MyModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._conv = keras.layers.Conv2D(8, 2)
        self._max_pool = keras.layers.MaxPool2D(2)
        self._flatten = keras.layers.Flatten()
        self._dense = keras.layers.Dense(2)
        self._softmax = keras.layers.Softmax()

    def call(self, inputs):
        x = self._conv(inputs)
        x = self._max_pool(x)
        x = self._flatten(x)
        x = self._dense(x)
        return self._softmax(x)

subclass_model = MyModel()

print(subclass_model(np.random.uniform(size=[1, 28, 28, 3])))
print(subclass_model.summary())


tensor([[0.5375, 0.4625]], grad_fn=<SoftmaxBackward0>)


Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                  │ ?                             │         104 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ max_pooling2d_3 (MaxPooling2D)     │ ?                             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ flatten_3 (Flatten)                │ ?                             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_4 (Dense)                    │ ?                             │       2,706 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ softmax_3 (Softmax)                │ ?                             │           0 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None


## Models are callable.
`keras.Model` instances are also callable. You can call models as if it is a function, no matter which way you used to build the model.

In [10]:
sample_data = np.random.uniform(size=[1, 28, 28, 3])

print(sequential_model(sample_data))
print(functional_model(sample_data))
print(subclass_model(sample_data))

tensor([[0.3852, 0.6148]], grad_fn=<SoftmaxBackward0>)
tensor([[0.5532, 0.4468]], grad_fn=<SoftmaxBackward0>)
tensor([[0.5829, 0.4171]], grad_fn=<SoftmaxBackward0>)


In [11]:
sliced_model = keras.Model(
    inputs=functional_model.inputs,
    outputs=functional_model.get_layer("conv2d_1").output,
)
sliced_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)         │ (None, 28, 28, 3)             │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ conv2d_1 (Conv2D)                  │ (None, 27, 27, 8)             │         104 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 104 (416.00 B)

 Trainable params: 104 (416.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model
Now we have defined the model. For actual training, we need the following pieces:

* Data
* Loss/Target function
* Optimizer

In [12]:

# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

In [13]:

# Let's use functional model defined above.
model = functional_model

In [14]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [15]:
model.fit(
    data,
    label,
    batch_size=5,
    epochs=2,
)

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7391 - sparse_categorical_accuracy: 0.3800 
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6861 - sparse_categorical_accuracy: 0.5333


## Save your model

In [23]:

# Save the weights.
model.save_weights("dummy.weights.h5")

In [24]:
sequential_model.load_weights("dummy.weights.h5")